In [13]:
import pandas as pd

import numpy as np

from sklearn import datasets

from sklearn.model_selection import GridSearchCV

from sklearn.svm import SVC

from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.model_selection import KFold

In [9]:
import json
import gc
import random
i = 0
reviews_lst_good= []
reviews_lst_bad = []
count_good_comments = 0
count_bad_comments = 0

# G:\\Мой диск\\Work\\Experiment\\Electronics_5.json
# /Volumes/GoogleDrive/Мой диск/Work/Experiment/Electronics_5.json
with open('G:\\Мой диск\\Work\\Experiment\\Electronics_5.json', 'r') as f: # указать свой путь  /Volumes/GoogleDrive/Мой диск/Work/Experiment/Books_5.json
        for line in f:
            if json.loads(line)['overall'] > 3:
                  count_good_comments += 1
            elif json.loads(line)['overall'] < 3:
                  count_bad_comments += 1
            
print (count_good_comments)
print (count_bad_comments)

1356067
190864


In [10]:
size_of_df = 5000 # min (count_good_comments, count_bad_comments)
print (size_of_df)

with open('G:\\Мой диск\\Work\\Experiment\\Electronics_5.json', 'r') as f: #/Volumes/GoogleDrive/Мой диск/Work/Experiment/Books_5.json
        for line in f:
            if json.loads(line)['overall'] > 3 and len(reviews_lst_good)<size_of_df:
                reviews_lst_good.append (json.loads(line))
            
            elif json.loads(line)['overall'] < 3 and len(reviews_lst_bad)<size_of_df:
                reviews_lst_bad.append (json.loads(line))

            if len(reviews_lst_good) + len (reviews_lst_bad) == 10000:
                break

random.shuffle(reviews_lst_good)
random.shuffle(reviews_lst_bad)
       

5000


In [11]:
print (len(reviews_lst_good))
print (len(reviews_lst_bad))

5000
5000


In [14]:
reviewerID_lst = []
reviewerName_lst = []
reviewText_lst = []
overall_lst = []
summary_lst = []

# reviews_lst [0]['overall']<5 #['asin']for 

for i in reviews_lst_good:
    reviewerID_lst.append (i['reviewerID'])
    reviewText_lst.append (i['reviewText'])
    overall_lst.append (i['overall'])
    summary_lst.append (i['summary'])
    
for i in reviews_lst_bad:
    reviewerID_lst.append (i['reviewerID'])
    reviewText_lst.append (i['reviewText'])
    overall_lst.append (i['overall'])
    summary_lst.append (i['summary'])
    
df = pd.DataFrame ()

df['reviewerID'] = reviewerID_lst
# df['reviewerName'] = reviewerName_lst
df['reviewText'] = reviewText_lst
df['overall'] = overall_lst
df['summary'] = summary_lst

In [15]:
df ['mark'] = -1
df.loc[df['overall'] > 3,'mark'] = 1

print (len(df[df['mark'] > 0]))
print (len(df[df['mark'] < 0]))

5000
5000


In [16]:
df[(df['mark'] != 0)&(df['mark'] != 1)]

,reviewerID,reviewText,overall,summary,mark
5000,A35T9TAWKLFU6T,"I bought these and the competing ""3M Electroni...",1.0,Container design flawed- wipes dried out in a ...,-1
5001,A3K7QVLK2AM5JU,I bought 4 packages of these and they were all...,1.0,dont use these for the philips cd recorder,-1
5002,A3JGIXQPX65TGJ,I needed extra speakers immediately for a last...,2.0,Cheap speakers,-1
5003,A20FHKCVWWI3EQ,Connected up my brand new Yamaha amp & Onkyo s...,1.0,Had a bad experience with this wire,-1
5004,A9NK4CE537Q0,"I bought this thinking since its expensive, it...",1.0,Red light came on,-1
5005,A2F7F3HULR9WCZ,Not going to ever order another tiffen item ev...,1.0,Bad flare.. Double image even!!,-1
5006,A255B9LGUY374U,I'm sure for a photographer with a still camer...,2.0,Don't like it,-1
5007,A29I1XIMOI4IV7,Although the item picture clearly shows batter...,2.0,Product does not come in a Duracell blister pack!,-1
5008,AV10H5K6VPQIJ,The outside is nice material but the sleeve in...,1.0,Decent,-1
5009,A3HFXZJ8KBT3OC,I've had this tripod for 4 years and have hand...,2.0,Broke,-1


In [21]:
reviews_train = list(df[df['overall'] > 3]['reviewText'][:-100]) + list(df[df['overall'] < 3]['reviewText'][:-100])
reviews_test = list(df[df['overall'] > 3]['reviewText'][-100:]) + list(df[df['overall'] < 3]['reviewText'][-100:])

In [22]:
import re
from nltk.corpus import stopwords 

REPLACE_NO_SPACE = re.compile("[.;:!\'?,\"()\[\]]")
REPLACE_WITH_SPACE = re.compile("(<br\s*/><br\s*/>)|(\-)|(\/)")

def preprocess_reviews(reviews):
    reviews = [REPLACE_NO_SPACE.sub("", line.lower()) for line in reviews]
    reviews = [REPLACE_WITH_SPACE.sub(" ", line) for line in reviews]
    
    return reviews

reviews_train_clean = preprocess_reviews(reviews_train)
reviews_test_clean = preprocess_reviews(reviews_test)

In [23]:
from sklearn import datasets

from sklearn.model_selection import GridSearchCV

from sklearn.svm import SVC

from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.model_selection import KFold

from sklearn.model_selection import train_test_split

In [24]:
y = list(df[df['overall'] > 3]['mark'][:-100]) + list(df[df['overall'] < 3]['mark'][:-100])

X = reviews_train_clean

X_train, X_val, y_train, y_val = train_test_split(
    X, y, train_size = 0.75
)


vectorizer = TfidfVectorizer() # создали генератор признаков 

vectorizer.fit_transform(X)

<9800x36646 sparse matrix of type '<class 'numpy.float64'>'
	with 690058 stored elements in Compressed Sparse Row format>

In [ ]:
# np.power(1.0, np.arange(-5, 6))

# np.array ([1])

In [28]:
grid = {'C': np.power(10.0, np.arange(-5, 6))}

cv = KFold(5, shuffle=True, random_state=241) # делим на 5 блоков

model = SVC(kernel='linear', random_state=241) # устанавливаем линейной ядро у SVM

gs = GridSearchCV(model, grid, scoring='accuracy', cv=cv) # показываем, чтобы использовал модель нашу 
# + искал параметр C сам из диапозона, метод для валивации KFold

gs.fit(vectorizer.transform(X), y)



C = gs.best_params_.get('C') # достаём оттуда только C
print (C)

1.0


In [29]:
# 4. Обучите SVM по всей выборке с оптимальным параметром C, найденным на предыдущем шаге.



model = SVC(kernel='linear', random_state=241, C)

model.fit(vectorizer.transform(X), y)

SyntaxError: positional argument follows keyword argument (<ipython-input-29-4459fcac58e6>, line 5)

In [ ]:
# 5. Найдите 10 слов с наибольшим по модулю весом. Они являются ответом на это задание. Укажите их через запятую или

# пробел, в нижнем регистре, в лексикографическом порядке.



words = vectorizer.get_feature_names()

coef = pd.DataFrame(model.coef_.data, model.coef_.indices)

top_words = coef[0].map(lambda w: abs(w)).sort_values(ascending=False).head(10).index.map(lambda i: words[i])

top_words.sort_values ()

# top_words

In [27]:
top_words

Index(['not', 'great', 'useless', 'best', 'poor', 'return', 'works',
       'returned', 'easy', 'excellent'],
      dtype='object')

In [5]:
# coding=utf-8
import pandas
import numpy as np
from sklearn import datasets #, grid_search
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import KFold

import sys
sys.path.append("..")
# from shad_util import print_answer

# 1. Загрузите объекты из новостного датасета 20 newsgroups, относящиеся к категориям "космос" и
# "атеизм" (инструкция приведена выше). Обратите внимание, что загрузка данных может занять несколько минут

newsgroups = datasets.fetch_20newsgroups(subset='all', categories=['alt.atheism', 'sci.space'])
X = newsgroups.data
y = newsgroups.target

# 2. Вычислите TF-IDF-признаки для всех текстов. Обратите внимание, что в этом задании мы предлагаем вам
# вычислить TF-IDF по всем данным. При таком подходе получается, что признаки на обучающем множестве используют
# информацию из тестовой выборки — но такая ситуация вполне законна, поскольку мы не используем значения целевой
# переменной из теста. На практике нередко встречаются ситуации, когда признаки объектов тестовой выборки известны на
# момент обучения, и поэтому можно ими пользоваться при обучении алгоритма.

vectorizer = TfidfVectorizer()
vectorizer.fit_transform(X)

# 3. Подберите минимальный лучший параметр C из множества [10^-5, 10^-4, ... 10^4, 10^5] для SVM с
# линейным ядром (kernel='linear') при помощи кросс-валидации по 5 блокам. Укажите параметр random_state=241 и для SVM,
# и для KFold. В качестве меры качества используйте долю верных ответов (accuracy).

grid = {'C': np.power(10.0, np.arange(-5, 6))}
cv = KFold(shuffle=True, random_state=241)
model = SVC(kernel='linear', random_state=241)
gs = GridSearchCV(model, grid, scoring='accuracy', cv=cv)
gs.fit(vectorizer.transform(X), y)

C = gs.best_params_.get('C')

# 4. Обучите SVM по всей выборке с оптимальным параметром C, найденным на предыдущем шаге.

model = SVC(kernel='linear', random_state=241, C=C)
model.fit(vectorizer.transform(X), y)

# 5. Найдите 10 слов с наибольшим по модулю весом. Они являются ответом на это задание. Укажите их через запятую или
# пробел, в нижнем регистре, в лексикографическом порядке.

words = vectorizer.get_feature_names()
coef = pandas.DataFrame(model.coef_.data, model.coef_.indices)
top_words = coef[0].map(lambda w: abs(w)).sort_values(ascending=False).head(10).index.map(lambda i: words[i])
top_words.sort()
# print_answer(1, ','.join(top_words))

/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:426: FutureWarning: You should specify a value for 'n_splits' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(NSPLIT_WARNING, FutureWarning)


TypeError: cannot sort an Index object in-place, use sort_values instead

In [7]:
np.shape (X)

(1786,)